### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2025, 1, 30)

In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-01-30
Yesterday: 2025-01-29


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-01-29'
ORDER BY a.name



(20, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-01-29' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-22,2025-01-29,5.15,5.25,176516896,5.25,5.10,B,SET,0.10,1.94
1,AIMIRT,2025-01-24,2025-01-29,10.50,10.50,2324512,10.50,10.40,B,SET,0.00,0.00
2,CPNREIT,2025-01-29,2025-01-29,12.40,12.50,20633284,12.50,11.90,B,SET,0.10,0.81
3,DIF,2025-01-23,2025-01-29,8.40,8.50,109420432,8.50,8.35,B,SET,0.10,1.19
4,GVREIT,2025-01-29,2025-01-29,6.50,6.60,622528,6.60,6.50,B,SET,0.10,1.54
5,JMART,2025-01-23,2025-01-29,11.20,11.90,56443408,11.90,11.20,B,SET50,0.70,6.25
6,KCE,2025-01-29,2025-01-29,22.70,22.50,25263468,23.20,22.50,B,SET100,-0.20,-0.88
7,MCS,2025-01-29,2025-01-29,6.35,6.25,387416,6.35,6.20,B,SET,-0.10,-1.57
8,NER,2025-01-29,2025-01-29,5.05,4.98,20083048,5.05,4.88,B,SET,-0.07,-1.39
9,SENA,2025-01-22,2025-01-29,2.20,2.18,17221776,2.20,2.16,B,SET,-0.02,-0.91


In [20]:
df.shape

(20, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,3BBIF,2025-01-22,2025-01-29,5.15,5.25,"176,516,896",5.25,5.10,1.94%,B
1,AIMIRT,2025-01-24,2025-01-29,10.50,10.50,"2,324,512",10.50,10.40,0.00%,B
2,CPNREIT,2025-01-29,2025-01-29,12.40,12.50,"20,633,284",12.50,11.90,0.81%,B
3,DIF,2025-01-23,2025-01-29,8.40,8.50,"109,420,432",8.50,8.35,1.19%,B
4,GVREIT,2025-01-29,2025-01-29,6.50,6.60,"622,528",6.60,6.50,1.54%,B
5,JMART,2025-01-23,2025-01-29,11.20,11.90,"56,443,408",11.90,11.20,6.25%,B
6,KCE,2025-01-29,2025-01-29,22.70,22.50,"25,263,468",23.20,22.50,-0.88%,B
7,MCS,2025-01-29,2025-01-29,6.35,6.25,"387,416",6.35,6.20,-1.57%,B
8,NER,2025-01-29,2025-01-29,5.05,4.98,"20,083,048",5.05,4.88,-1.39%,B
9,SENA,2025-01-22,2025-01-29,2.20,2.18,"17,221,776",2.20,2.16,-0.91%,B


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(20, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(182, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(20, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,3BBIF,2025-01-22,2025-01-29,5.15,5.25,176516896,5.25,5.10,1.94,B,5.25,0.00,4634606,2025-01-30
1,AIMIRT,2025-01-24,2025-01-29,10.50,10.50,2324512,10.50,10.40,0.00,B,10.40,-0.10,51318,2025-01-30
2,CPNREIT,2025-01-29,2025-01-29,12.40,12.50,20633284,12.50,11.90,0.81,B,12.50,0.00,742243,2025-01-30
3,DIF,2025-01-23,2025-01-29,8.40,8.50,109420432,8.50,8.35,1.19,B,8.40,-0.10,11588460,2025-01-30
4,GVREIT,2025-01-29,2025-01-29,6.50,6.60,622528,6.60,6.50,1.54,B,6.65,0.05,1165744,2025-01-30
5,JMART,2025-01-23,2025-01-29,11.20,11.90,56443408,11.90,11.20,6.25,B,11.30,-0.60,5039550,2025-01-30
6,KCE,2025-01-29,2025-01-29,22.70,22.50,25263468,23.20,22.50,-0.88,B,22.20,-0.30,4659758,2025-01-30
7,MCS,2025-01-29,2025-01-29,6.35,6.25,387416,6.35,6.20,-1.57,B,6.25,0.00,23301,2025-01-30
8,NER,2025-01-29,2025-01-29,5.05,4.98,20083048,5.05,4.88,-1.39,B,4.96,-0.02,3239382,2025-01-30
9,SENA,2025-01-22,2025-01-29,2.20,2.18,17221776,2.20,2.16,-0.91,B,2.18,0.00,276070,2025-01-30


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
3BBIF    2025-01-22 2025-01-30   5.15   5.25   1.94% 176,516,896   5.25   5.10 B Update
AIMIRT   2025-01-24 2025-01-30  10.50  10.40  -0.95%   2,324,512  10.50  10.40 B Update
AWC      2025-01-29 2025-01-30   3.20   3.18  -0.63%  98,706,804   3.20   3.08 S Update
BCH      2025-01-30 2025-01-30  14.20  14.30    0.7%  82,658,432  15.20  14.20 S Insert
CPNREIT  2025-01-29 2025-01-30  12.40  12.50   0.81%  20,633,284  12.50  11.90 B Update
DIF      2025-01-30 2025-01-30   8.50   8.40  -1.18% 109,420,432   8.50   8.35 B Insert
GVREIT   2025-01-29 2025-01-30   6.50   6.65   2.31%     622,528   6.65   6.50 B Update
JMART    2025-01-30 2025-01-30  11.90  11.30  -5.04%  56,443,408  11.90  11.20 B Insert
JMT      2025-01-30 2025-01-30  15.80  15.30  -3.16% 381,337,088  15.80  14.70 S Insert
KCE      2025-01-29 2025-01-30  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [56]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-01-30' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [58]:
df = pd.read_sql(sql, conlite)
df.shape

(20, 10)

In [60]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-22,2025-01-30,5.15,5.25,"353,033,792",5.25,5.10,B,SET,0.10,1.94%
1,AIMIRT,2025-01-24,2025-01-30,10.50,10.40,"4,649,024",10.50,10.40,B,SET,-0.10,-0.95%
2,CPNREIT,2025-01-29,2025-01-30,12.40,12.50,"41,266,568",12.50,11.90,B,SET,0.10,0.81%
3,DIF,2025-01-30,2025-01-30,8.50,8.40,"109,420,432",8.50,8.35,B,SET,-0.10,-1.18%
4,GVREIT,2025-01-29,2025-01-30,6.50,6.65,"1,245,056",6.65,6.50,B,SET,0.15,2.31%
5,JMART,2025-01-30,2025-01-30,11.90,11.30,"56,443,408",11.90,11.20,B,SET50,-0.60,-5.04%
6,KCE,2025-01-29,2025-01-30,22.70,22.20,"50,526,936",23.20,22.20,B,SET100,-0.50,-2.20%
7,MCS,2025-01-29,2025-01-30,6.35,6.25,"774,832",6.35,6.20,B,SET,-0.10,-1.57%
8,NER,2025-01-29,2025-01-30,5.05,4.96,"40,166,096",5.05,4.88,B,SET,-0.09,-1.78%
9,SENA,2025-01-22,2025-01-30,2.20,2.18,"34,443,552",2.20,2.16,B,SET,-0.02,-0.91%


In [62]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [64]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
5,JMART,2025-01-30,2025-01-30,11.90,11.30,"56,443,408",11.90,11.20,B,SET50,-0.60,-5.04%


In [66]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [68]:
conlite.close()